In [1]:
from src.inference.gemini import ChatGemini
from src.inference.ollama import ChatOllama
from src.message import SystemMessage,HumanMessage
from dotenv import load_dotenv
import os

load_dotenv()
api_key=os.getenv('GOOGLE_API_KEY')
llm=ChatGemini(model='gemini-2.0-flash-exp',api_key=api_key,temperature=0)

In [2]:
# llm=ChatOllama(model='deepseek-r1:8b',temperature=0.6)

In [3]:
plan_prompt='''
Generate the comprehensive plan for developing the software application according to the  user's desired specifications. 
Your primary objective is to understand the app's logic, purpose, and it's functionality, then create a detailed plan for implementing it.

## Instructions
- Be clear, concise, and avoid ambiguity in your explanations.
- Don't include anything about testing or deployment of the app, in the plan.
- Use a modular and scalable design, ensuring the app can be expanded or modified easily.
- For now make the plan for making python based apps.
- Don't include code in the plan.
'''

In [4]:
from pydantic import BaseModel,Field

class Plan(BaseModel):
    title:str=Field(...,description='The name of the app to develop',example=['Calculator'])
    overview:str=Field(...,description='A high-level description of the app',example=['A simple calculator for basic arithmetic operations.'])
    requirements:str=Field(...,description='The user requirements of the app',example=['I want to be able to add, subtract, multiply, and divide two numbers.'])
    logic:str=Field(...,description='The logic about the working of the app',example=['Supports addition, subtraction, multiplication, and division.'])
    libraries:list[str]=Field(description='The python libraries needed for implementing the app',example=[['streamlit','numpy','pandas']],default_factory=[])
    plan:list[str]=Field(...,description='The plan for developing the app',example=['Design the user interface. Develop the backend logic.'])


In [5]:
response=llm.invoke([SystemMessage(plan_prompt),HumanMessage('Create a tic tac toe')],model=Plan)

{
  "title": "Tic Tac Toe",
  "overview": "A classic two-player game of Tic Tac Toe.",
  "requirements": "The game should allow two players to take turns marking spaces on a 3x3 grid. The first player to get three of their marks in a row (horizontally, vertically, or diagonally) wins. If all spaces are filled and no player has won, the game is a draw.",
  "logic": "The game uses a 3x3 grid represented as a list of lists. Players take turns inputting their moves. The game checks for a win after each move. If no win is detected and the board is full, the game is a draw.",
  "libraries": [
    "tkinter"
  ],
  "plan": [
    "Set up the game board using tkinter.",
    "Create a function to handle player moves.",
    "Implement a function to check for a win.",
    "Implement a function to check for a draw.",
    "Create a main game loop to handle turns and game state.",
    "Add a reset button to start a new game."
  ]
}


In [6]:
print(response.model_dump_json(indent=2))

{
  "title": "Tic Tac Toe",
  "overview": "A classic two-player game of Tic Tac Toe.",
  "requirements": "The game should allow two players to take turns marking spaces on a 3x3 grid. The first player to get three of their marks in a row (horizontally, vertically, or diagonally) wins. If all spaces are filled and no player has won, the game is a draw.",
  "logic": "The game uses a 3x3 grid represented as a list of lists. Players take turns inputting their moves. The game checks for a win after each move. If no win is detected and the board is full, the game is a draw.",
  "libraries": [
    "tkinter"
  ],
  "plan": [
    "Set up the game board using tkinter.",
    "Create a function to handle player moves.",
    "Implement a function to check for a win.",
    "Implement a function to check for a draw.",
    "Create a main game loop to handle turns and game state.",
    "Add a reset button to start a new game."
  ]
}


In [7]:
plan_template=f'''
# {response.title}

{response.overview}

## Requirements

{response.requirements}

## Logic

{response.logic}

## Python libraries

{'\n'.join([f'- {library}' for library in response.libraries])}

## Plan

{'\n'.join(f'- {step}' for step in response.plan)}
'''

In [8]:
from IPython.display import Markdown

In [9]:
Markdown(plan_template)


# Tic Tac Toe

A classic two-player game of Tic Tac Toe.

## Requirements

The game should allow two players to take turns marking spaces on a 3x3 grid. The first player to get three of their marks in a row (horizontally, vertically, or diagonally) wins. If all spaces are filled and no player has won, the game is a draw.

## Logic

The game uses a 3x3 grid represented as a list of lists. Players take turns inputting their moves. The game checks for a win after each move. If no win is detected and the board is full, the game is a draw.

## Python libraries

- tkinter

## Plan

- Set up the game board using tkinter.
- Create a function to handle player moves.
- Implement a function to check for a win.
- Implement a function to check for a draw.
- Create a main game loop to handle turns and game state.
- Add a reset button to start a new game.


In [10]:
from pydantic import BaseModel,Field


class File(BaseModel):
    name: str=Field(...,description='The name of the file',example=['main.py','logic.py','snake.py'])
    description: str=Field(...,description='A brief explanation of the file’s purpose, its role in the app, and how it contributes to the overall functionality.')
    file_path:str=Field(...,description='The path of the file',example=['./app_name/subfolder1/abc.py'])
    imports: list[str]=Field(description='The libraries or modules in the app to be imported in the file',example=[['Tool','pandas','Food']],default_factory=[])
    classes: list['Class']=Field(description='The classes present in the file',default_factory=[])
    variables: list[str]=Field(description='The variables present in the file',examples=[['constant','settings']],default_factory=[])
    
class Function(BaseModel):
    name:str=Field(...,description='The name of the function',examples=['move_snake'])
    parameters:list[str]=Field(description='The parameters the of the function',examples=[['snake:Snake','direction:str']],default_factory=[])
    description:str=Field(...,description='The purpose of the function in detail',examples=['This function tells the snake to mov'])
    return_type:str=Field(description='The return type of the function', examples=['int','list[str]','Move'],default=None)

class Class(BaseModel):
    name:str=Field(description='The name of the class',examples=['Snake'])
    properties:list[str]=Field(description='The properties present in the class',examples=['size','file_size'],default=[])
    methods:list[Function]=Field(description='The functions present in the class',default_factory=[])

class Folder(BaseModel):
    name: str = Field(..., description="The name of the folder or the module", examples=["app"])
    files: list[File] = Field(..., description="Files directly inside the current folder")
    subfolders: list["Folder"] = Field(..., description="It contains further subfolders and files")

    class Config:
        # Pydantic needs this for recursive models
        arbitrary_types_allowed = True
        from_attributes = True


In [11]:
structure_prompt='''
Generate the file structure of the app based upon the plan provided.

## Instructions
- Ensure the file contents and structure are consistent with modern best practices for software development using python.
- Use a modular and scalable design, ensuring the app can be expanded or modified easily.
- Each subfolder has their own respective files for implementating that particular logic of the app.
- Include `main.py` in the root directory of the app, it is this file the user will run, launch the app.
- Adapt the structure and components based on the specific requirements of the app described by the user.
- If needed include the requirements.txt file in the root directory.
- Each module has its own `__init__.py` file.
'''

In [12]:
response=llm.invoke([SystemMessage(structure_prompt),HumanMessage(plan_template)],model=Folder)

{
  "name": "tic_tac_toe",
  "files": [
    {
      "name": "main.py",
      "description": "This is the main entry point of the application. It initializes the game and starts the main game loop.",
      "file_path": "./main.py",
      "imports": [
        "tkinter",
        "game"
      ],
      "classes": [],
      "variables": []
    },
    {
      "name": "game.py",
      "description": "This file contains the main game logic, including the game board, player moves, win conditions, and draw conditions.",
      "file_path": "./game/game.py",
      "imports": [
        "tkinter",
        "tkinter.messagebox"
      ],
      "classes": [
        {
          "name": "TicTacToe",
          "properties": [
            "board",
            "current_player",
            "buttons"
          ],
          "methods": [
            {
              "name": "__init__",
              "parameters": [
                "self",
                "master"
              ],
              "description": "Ini

In [13]:
print(response.model_dump_json(indent=2))

{
  "name": "tic_tac_toe",
  "files": [
    {
      "name": "main.py",
      "description": "This is the main entry point of the application. It initializes the game and starts the main game loop.",
      "file_path": "./main.py",
      "imports": [
        "tkinter",
        "game"
      ],
      "classes": [],
      "variables": []
    },
    {
      "name": "game.py",
      "description": "This file contains the main game logic, including the game board, player moves, win conditions, and draw conditions.",
      "file_path": "./game/game.py",
      "imports": [
        "tkinter",
        "tkinter.messagebox"
      ],
      "classes": [
        {
          "name": "TicTacToe",
          "properties": [
            "board",
            "current_player",
            "buttons"
          ],
          "methods": [
            {
              "name": "__init__",
              "parameters": [
                "self",
                "master"
              ],
              "description": "Ini

In [14]:
from pydantic import BaseModel,Field

class Script(BaseModel):
    file_name: str=Field(...,description='The name of the file',examples=['main.py'])
    file_path: str=Field(...,description='The path of the file',examples=['./app_name/main.py'])
    code_block: str=Field(...,description='The python script goes here',examples=['''print("Hello World")'''])

In [15]:
develop_prompt='''
Generate the python script for the provided file of the app. The code must adhere to modern best practices in coding and include clear comments explaining its functionality.

## Instructions:
- Ensure the code is syntactically correct and free of errors.
- Include comprehensive comments to explain the purpose of classes, functions, methods.
- Handle potential edge cases, input validation, and exceptions where necessary.
- Ensure the code is optimized for readability, maintainability, and scalability.
'''

In [16]:
model=response.files[0]
filename=model.name
description=model.description
imports=model.imports
classes=model.classes
variables=model.variables

In [17]:
user_prompt=f'''
### Filename: 
{filename}
### Description: 
{description}
Imports in the File: 
{"\n".join([f'- {import_}' for import_ in imports])}
Classes in the File: 
{"\n".join([f'- {class_}' for class_ in classes])}
Variables in the File: 
{"\n".join([f'- {variable_}' for variable_ in variables])}

Write the Python script for this file.
'''

In [18]:
response=llm.invoke([SystemMessage(develop_prompt),HumanMessage(user_prompt)],model=Script)

{
  "file_name": "main.py",
  "file_path": "./main.py",
  "code_block": "# main.py\n\nimport tkinter as tk\nfrom game import Game  # Assuming game.py contains the Game class\n\n\n# Main function to initialize and run the game\ndef main():\n    # Create the main application window\n    root = tk.Tk()\n    root.title(\"My Awesome Game\")  # Set the title of the window\n\n    # Initialize the game with the root window\n    game = Game(root)\n\n    # Start the main game loop\n    root.mainloop()\n\n\n# Entry point of the script\nif __name__ == \"__main__\":\n    main()\n"
}


In [13]:
content='''
<StructureContent>
    <Folder>
        <Name type="str">calculator_app</Name>
        <Files type="list[File]">
            <File>
                <Name type="str">main.py</Name>
                <Description type="str">The main entry point of the application. It initializes the GUI and starts the event loop.</Description>
                <Path type="str">./calculator_app/main.py</Path>
                <Imports type="list[str]">
                    <ImportItem>tkinter</ImportItem>
                    <ImportItem>calculator_app.calculator</ImportItem>
                    <ImportItem>calculator_app.gui</ImportItem>
                </Imports>
                <Classes type="list[Class]"/>
                <Variables type="list[str]"/>
            </File>
            <File>
                <Name type="str">requirements.txt</Name>
                <Description type="str">Lists the dependencies required to run the application.</Description>
                <Path type="str">./calculator_app/requirements.txt</Path>
                <Imports type="list[str]"/>
                <Classes type="list[Class]"/>
                <Variables type="list[str]"/>
            </File>
        </Files>
        <Subfolders type="list[Folder]">
            <Folder>
                <Name type="str">calculator_app</Name>
                <Files type="list[File]">
                    <File>
                        <Name type="str">__init__.py</Name>
                        <Description type="str">Marks the 'calculator_app' directory as a Python package.</Description>
                        <Path type="str">./calculator_app/calculator_app/__init__.py</Path>
                        <Imports type="list[str]"/>
                        <Classes type="list[Class]"/>
                        <Variables type="list[str]"/>
                    </File>
                </Files>
                <Subfolders type="list[Folder]">
                    <Folder>
                        <Name type="str">calculator</Name>
                        <Files type="list[File]">
                            <File>
                                <Name type="str">__init__.py</Name>
                                <Description type="str">Marks the 'calculator' directory as a Python package.</Description>
                                <Path type="str">./calculator_app/calculator_app/calculator/__init__.py</Path>
                                <Imports type="list[str]"/>
                                <Classes type="list[Class]"/>
                                <Variables type="list[str]"/>
                            </File>
                            <File>
                                <Name type="str">operations.py</Name>
                                <Description type="str">Implements the arithmetic operations (addition, subtraction, multiplication, division).</Description>
                                <Path type="str">./calculator_app/calculator_app/calculator/operations.py</Path>
                                <Imports type="list[str]"/>
                                <Classes type="list[Class]"/>
                                <Variables type="list[str]"/>
                            </File>
                            <File>
                                <Name type="str">calculator.py</Name>
                                <Description type="str">Contains the main calculator logic, handling input and calling the appropriate operations.</Description>
                                <Path type="str">./calculator_app/calculator_app/calculator/calculator.py</Path>
                                <Imports type="list[str]">
                                    <ImportItem>calculator_app.calculator.operations</ImportItem>
                                </Imports>
                                <Classes type="list[Class]">
                                    <Class>
                                        <Name type="str">Calculator</Name>
                                        <Properties type="list[str]"/>
                                        <Methods type="list[Method]">
                                            <Method>
                                                <Name type="str">add</Name>
                                                <Parameters type="list[str]">
                                                    <ParameterItem type="str">x: float</ParameterItem>
                                                    <ParameterItem type="str">y: float</ParameterItem>
                                                </Parameters>
                                                <Description type="str">Adds two numbers.</Description>
                                                <ReturnType type="str">float</ReturnType>
                                            </Method>
                                            <Method>
                                                <Name type="str">subtract</Name>
                                                <Parameters type="list[str]">
                                                    <ParameterItem type="str">x: float</ParameterItem>
                                                    <ParameterItem type="str">y: float</ParameterItem>
                                                </Parameters>
                                                <Description type="str">Subtracts two numbers.</Description>
                                                <ReturnType type="str">float</ReturnType>
                                            </Method>
                                            <Method>
                                                <Name type="str">multiply</Name>
                                                <Parameters type="list[str]">
                                                    <ParameterItem type="str">x: float</ParameterItem>
                                                    <ParameterItem type="str">y: float</ParameterItem>
                                                </Parameters>
                                                <Description type="str">Multiplies two numbers.</Description>
                                                <ReturnType type="str">float</ReturnType>
                                            </Method>
                                            <Method>
                                                <Name type="str">divide</Name>
                                                <Parameters type="list[str]">
                                                    <ParameterItem type="str">x: float</ParameterItem>
                                                    <ParameterItem type="str">y: float</ParameterItem>
                                                </Parameters>
                                                <Description type="str">Divides two numbers, handling division by zero.</Description>
                                                <ReturnType type="str">float</ReturnType>
                                            </Method>
                                        </Methods>
                                    </Class>
                                </Classes>
                                <Variables type="list[str]"/>
                            </File>
                        </Files>
                        <Subfolders type="list[Folder]"/>
                    </Folder>
                    <Folder>
                        <Name type="str">gui</Name>
                        <Files type="list[File]">
                            <File>
                                <Name type="str">__init__.py</Name>
                                <Description type="str">Marks the 'gui' directory as a Python package.</Description>
                                <Path type="str">./calculator_app/calculator_app/gui/__init__.py</Path>
                                <Imports type="list[str]"/>
                                <Classes type="list[Class]"/>
                                <Variables type="list[str]"/>
                            </File>
                            <File>
                                <Name type="str">calculator_gui.py</Name>
                                <Description type="str">Defines the GUI elements and their layout using Tkinter. It also handles user input and displays the results.</Description>
                                <Path type="str">./calculator_app/calculator_app/gui/calculator_gui.py</Path>
                                <Imports type="list[str]">
                                    <ImportItem>tkinter</ImportItem>
                                    <ImportItem>calculator_app.calculator.calculator</ImportItem>
                                </Imports>
                                <Classes type="list[Class]">
                                    <Class>
                                        <Name type="str">CalculatorGUI</Name>
                                        <Properties type="list[str]">
                                            <PropertyItem type="str">calculator</PropertyItem>
                                            <PropertyItem type="str">root</PropertyItem>
                                            <PropertyItem type="str">entry</PropertyItem>
                                        </Properties>
                                        <Methods type="list[Method]">
                                            <Method>
                                                <Name type="str">__init__</Name>
                                                <Parameters type="list[str]">
                                                    <ParameterItem type="str">root: tk.Tk</ParameterItem>
                                                </Parameters>
                                                <Description type="str">Initializes the CalculatorGUI with a Tkinter root window.</Description>
                                                <ReturnType type="str">None</ReturnType>
                                            </Method>
                                            <Method>
                                                <Name type="str">create_widgets</Name>
                                                <Parameters type="list[str]"/>
                                                <Description type="str">Creates and arranges the GUI widgets (buttons, entry field).</Description>
                                                <ReturnType type="str">None</ReturnType>
                                            </Method>
                                            <Method>
                                                <Name type="str">button_click</Name>
                                                <Parameters type="list[str]">
                                                    <ParameterItem type="str">number: str</ParameterItem>
                                                </Parameters>
                                                <Description type="str">Handles button clicks, appending the clicked number to the entry field.</Description>
                                                <ReturnType type="str">None</ReturnType>
                                            </Method>
                                            <Method>
                                                <Name type="str">button_clear</Name>
                                                <Parameters type="list[str]"/>
                                                <Description type="str">Clears the entry field.</Description>
                                                <ReturnType type="str">None</ReturnType>
                                            </Method>
                                            <Method>
                                                <Name type="str">button_equal</Name>
                                                <Parameters type="list[str]"/>
                                                <Description type="str">Evaluates the expression in the entry field and displays the result.</Description>
                                                <ReturnType type="str">None</ReturnType>
                                            </Method>
                                        </Methods>
                                    </Class>
                                </Classes>
                                <Variables type="list[str]"/>
                            </File>
                        </Files>
                        <Subfolders type="list[Folder]"/>
                    </Folder>
                </Subfolders>
            </Folder>
        </Subfolders>
    </Folder>
</StructureContent>
'''

In [7]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Method(BaseModel):
    name: str
    parameters: List[str] = Field(default_factory=list)
    description: str
    return_type: str

class Class(BaseModel):
    name: str
    properties: List[str] = Field(default_factory=list)
    methods: List[Method] = Field(default_factory=list)

class File(BaseModel):
    name: str
    description: str
    path: str
    imports: List[str] = Field(default_factory=list)
    classes: List[Class] = Field(default_factory=list)
    variables: List[str] = Field(default_factory=list)

class Folder(BaseModel):
    name: str
    files: List[File] = Field(default_factory=list)
    subfolders: List["Folder"] = Field(default_factory=list)

    class Config:
        arbitrary_types_allowed = True

class Structure(BaseModel):
    folder: Folder

In [8]:
from xml.etree import ElementTree as ET

def parse_method(method_element):
    """Parses a <Method> XML element into a Method Pydantic object."""
    return Method(
        name=method_element.find("Name").text,
        parameters=[param.text for param in method_element.findall("Parameters/ParameterItem")],
        description=method_element.find("Description").text,
        return_type=method_element.find("ReturnType").text
    )

def parse_class(class_element):
    """Parses a <Class> XML element into a Class Pydantic object."""
    return Class(
        name=class_element.find("Name").text,
        properties=[prop.text for prop in class_element.findall("Properties/PropertyItem")],
        methods=[parse_method(method) for method in class_element.findall("Methods/Method")]
    )

def parse_file(file_element):
    """Parses a <File> XML element into a File Pydantic object."""
    return File(
        name=file_element.find("Name").text,
        description=file_element.find("Description").text,
        path=file_element.find("Path").text,
        imports=[imp.text for imp in file_element.findall("Imports/ImportItem")],
        classes=[parse_class(cls) for cls in file_element.findall("Classes/Class")],
        variables=[var.text for var in file_element.findall("Variables/VariableItem")]
    )

def parse_folder(folder_element):
    """Parses a <Folder> XML element into a Folder Pydantic object."""
    return Folder(
        name=folder_element.find("Name").text,
        files=[parse_file(file) for file in folder_element.findall("Files/File")],
        subfolders=[parse_folder(subfolder) for subfolder in folder_element.findall("Subfolders/Folder")]
    )

def xml_to_pydantic(xml_string):
    """Converts XML string into a Pydantic Structure object."""
    root = ET.fromstring(xml_string)
    folder = parse_folder(root.find("Folder"))
    return Structure(folder=folder)

In [14]:
print(xml_to_pydantic(content).model_dump_json(indent=2))

{
  "folder": {
    "name": "calculator_app",
    "files": [
      {
        "name": "main.py",
        "description": "The main entry point of the application. It initializes the GUI and starts the event loop.",
        "path": "./calculator_app/main.py",
        "imports": [
          "tkinter",
          "calculator_app.calculator",
          "calculator_app.gui"
        ],
        "classes": [],
        "variables": []
      },
      {
        "name": "requirements.txt",
        "description": "Lists the dependencies required to run the application.",
        "path": "./calculator_app/requirements.txt",
        "imports": [],
        "classes": [],
        "variables": []
      }
    ],
    "subfolders": [
      {
        "name": "calculator_app",
        "files": [
          {
            "name": "__init__.py",
            "description": "Marks the 'calculator_app' directory as a Python package.",
            "path": "./calculator_app/calculator_app/__init__.py",
            "impo

In [1]:
from src.agent.engineer.structure.utils import show_tree
folder_structure = {
    "folder": {
        "name": "calculator_app",
        "files": [
            {"name": "main.py"},
            {"name": "requirements.txt"}
        ],
        "subfolders": [
            {
                "name": "calculator_app",
                "files": [
                    {"name": "__init__.py"}
                ],
                "subfolders": [
                    {
                        "name": "calculator",
                        "files": [
                            {"name": "__init__.py"},
                            {"name": "operations.py"},
                            {"name": "calculator.py"}
                        ],
                        "subfolders": []
                    },
                    {
                        "name": "gui",
                        "files": [
                            {"name": "__init__.py"},
                            {"name": "calculator_gui.py"}
                        ],
                        "subfolders": []
                    }
                ]
            }
        ]
    }
}

# Call the function to display the structure
show_tree(folder_structure)


📂 calculator_app
   📄 main.py
   📄 requirements.txt
   📂 calculator_app
      📄 __init__.py
      📂 calculator
         📄 __init__.py
         📄 operations.py
         📄 calculator.py
      📂 gui
         📄 __init__.py
         📄 calculator_gui.py
